In [88]:
import pypsa
import pandas as pd


runs = ["Business_As_Usual",
        "LTS_Baseline_DAC",
        "Tech_DAC",
        "Life_DAC",
        "Tango_DAC"
        ]

In [206]:
co2

,cluster,type,noscaling,Past_trends,LTS_Baseline,Tech,Life,Tango
subtype,,,,,,,,
DAC0,co2,links,-1.125648e+03,-1.523062e+03,-8.845708e+02,-1.799294e+02,-1.768943e+02,-6.931649e-06
OCGT2,co2,links,3.294235e-05,8.358303e-05,5.369283e-05,1.542115e+00,7.049727e-01,2.315544e+00
SMR CC2,co2,links,3.416201e-01,1.026827e-01,5.258072e+00,1.366542e+01,2.088469e+00,1.387546e+01
SMR2,co2,links,2.959003e-05,6.385709e-05,3.987239e-05,1.207096e-05,4.770892e-05,1.389525e-01
biogas to gas2,co2,links,-3.908255e-07,-9.028748e-07,-6.639715e-07,-2.601803e-07,-5.276778e-07,-2.508395e-08
co2 vent1,co2,links,7.558007e-07,2.995969e-06,1.117385e-06,5.021771e-07,2.052937e-06,6.583545e-07
gas for industry CC2,co2,links,4.236556e+00,3.941111e+00,4.134222e+00,2.736556e+00,2.570333e+00,8.540125e-08
gas for industry2,co2,links,9.206388e-07,2.014161e-06,1.168936e-06,3.049271e-07,1.417595e-06,2.888037e+01
process emissions CC1,co2,links,7.640000e+00,7.014000e+00,6.673000e+00,5.054500e+00,4.210500e+00,3.176803e+00


In [89]:
def df_select(df,col,vals):
    return df.loc[[i for i in df.index if df.loc[i][col] in vals]]

In [246]:
for run in runs:    
    df = pd.read_csv("results/"+run+"/csvs/supply_energy.csv")
    n1=df.iloc[2,3]
    n2=df.iloc[2,4]
    df=df.drop([0,1,2,3])
    df.columns=['cluster','type','subtype',n1,n2]
    df_co2 = df_select(df,'cluster',['co2','co2 stored'])
    df_co2[[n1,n2]]=df_co2[[n1,n2]].astype('float64') /1e6
    df_co2=df_co2.set_index('subtype')
    if run == runs[0]:
        co2 = df_co2
    else:
        if n1=='noscaling':
            name=n2
        else:
            name=n1
        co2 = pd.concat([co2,df_co2[name]],axis=1)
    
co2=co2.fillna(0)
#df_co2


In [199]:
links_loads = co2.loc[[i for i in co2.index if co2.loc[i]['type'] in ['links','loads']]]
links_loads.sum()

cluster         co2co2co2co2co2co2co2co2co2co2co2co2co2co2co2c...
type            linkslinkslinkslinkslinkslinkslinkslinkslinksl...
noscaling                                             2136.664345
Past_trends                                           2637.153765
LTS_Baseline                                          1855.772541
Tech                                                  1030.609003
Life                                                   922.716563
Tango                                                  597.241201
dtype: object

In [120]:
# avoidable emissions
cats=['OCGT2','gas boiler2','urban central gas CHP3']
co2_avoidable_all = co2.loc[[i for i in co2.index if any(c in i for c in cats)]]
co2_avoidable = co2_avoidable_all.sum()
co2_avoidable

cluster                       co2co2co2co2co2co2co2
type            linkslinkslinkslinkslinkslinkslinks
noscaling                                289.058429
Past_trends                              269.317562
LTS_Baseline                              223.18569
Tech                                     115.452055
Life                                     180.513433
Tango                                    196.492442
dtype: object

In [147]:
## All CO2 unaccounted emissions
co2_gross = df_select(co2,'type',['stores']).sum()
co2_gross

cluster         co2co2 stored
type             storesstores
noscaling        -2136.664345
Past_trends      -2637.153765
LTS_Baseline     -1855.772541
Tech             -1030.609003
Life              -922.716563
Tango             -597.241201
dtype: object

In [151]:
co2_unavoidable = co2_gross+co2_avoidable
co2_unavoidable=co2_unavoidable.drop(['cluster','type'])

In [185]:
cc = co2.loc[[i for i in co2.index if "CC" in i]]
cc.sum()

cluster         co2co2co2co2co2co2co2 storedco2 storedco2 stor...
type            linkslinkslinkslinkslinkslinkslinkslinkslinksl...
noscaling                                              512.568485
Past_trends                                            619.548163
LTS_Baseline                                           470.958632
Tech                                                   348.518068
Life                                                   232.048967
Tango                                                  202.290669
dtype: object

INFO:pypsa.io:Imported network elec_sc_Tango_s_37_lvopt__Co2L0p05-96H-T-H-B-I-A-solar+p3-dist1_2050.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [127]:
scens = ["Past_trends",  #: {"pop": 2.1, 'pkm': 1, 'passmode': 1,   'occ': 1, 'freight': 1,'freightmode': 1,   "air_dom": air_pt,"air_eu": air_pt,"air_intl": air_pt},
            'LTS_Baseline' ,  #: {'pop': 1.5, 'pkm': 1, 'passmode': 1.9, 'occ': 1, 'freight': 1,'freightmode': 1.7, "air_dom": air_pt,"air_eu": air_pt,"air_intl": air_pt},
            'Tango'        ,  #: {'pop': 1.5, 'pkm': 4, 'passmode': 4  , 'occ': 4, 'freight': 4,'freightmode': 4,   "air_dom": 0,     "air_eu": 0.8,   "air_intl": 1},
            'Tech'         ,  #: {'pop': 1.5, 'pkm': 1, 'passmode': 1.9, 'occ': 1, 'freight': 1,'freightmode': 1.7, "air_dom": air_pt,"air_eu": air_pt,"air_intl": air_pt},
            'Life'           #: {'pop': 1.5, 'pkm': 4, 'passmode': 4  , 'occ': 4, 'freight': 4,'freightmode': 1.7, "air_dom": 0,     "air_eu": 0.8,   "air_intl": 1}
            ]

In [145]:
afolu = pd.DataFrame(index=scens, columns = ['fuels','land use','agriculture',"net"])

for s in scens:
    nrg = pd.read_csv("../../../Documents/eucalc_scenarios/"+s+"/afolu/co2-emissions-per-source.csv",index_col=0).loc[2050,"Fuel"]
    lu_em = pd.read_csv("../../../Documents/eucalc_scenarios/"+s+"/afolu/greenhouse-gas-emissions.csv",index_col=0).loc[2050,"Land use.1"]
    ag_em = pd.read_csv("../../../Documents/eucalc_scenarios/"+s+"/afolu/greenhouse-gas-emissions.csv",index_col=0).loc[2050,"Agriculture"]
    afolu.loc[s]['fuels'] = nrg
    afolu.loc[s]['land use'] = lu_em
    afolu.loc[s]['agriculture'] = ag_em
    afolu.loc[s]['net'] = ag_em - nrg + lu_em



In [146]:
afolu

,fuels,land use,agriculture,net
Past_trends,51.352023,-374.563843,350.755591,-75.160275
LTS_Baseline,54.386017,-338.812073,402.175025,8.976934
Tango,40.026734,-746.406214,180.006743,-606.426206
Tech,44.750194,-418.286908,376.250952,-86.786151
Life,39.304755,-747.155582,187.72363,-598.736707


In [248]:
net_emissions = afolu['net']+co2_gross*-1 # co2_unavoidable
net_emissions

LTS_Baseline    1864.749476
Life             323.979856
Past_trends      2561.99349
Tango             -9.185005
Tech             943.822852
cluster                 NaN
noscaling               NaN
type                    NaN
dtype: object

In [245]:
# get carbon flows by type
# 1. atmosphere

# 2. stores
co2_stores_all = co2.loc[[i for i in co2.index if co2.loc[i]["cluster"]=="co2 stored"]]
co2_stores = pd.DataFrame(columns=co2.columns[2:])
co2_stores.loc['CCS'] = co2_stores_all.loc[[i for i in co2_stores_all.index if 'CC' in i and not 'bio' in i]].drop(['cluster','type'],axis=1).sum()
co2_stores.loc['BECCS'] = co2_stores_all.loc[[i for i in co2_stores_all.index if 'CC' in i and 'bio' in i]].drop(['cluster','type'],axis=1).sum()
co2_stores.loc['SMR CC'] = co2_stores_all.loc[[i for i in co2_stores_all.index if 'SMR' in i]].drop(['cluster','type'],axis=1).sum()
co2_stores.loc['chemical production'] = co2_stores_all.loc[[i for i in co2_stores_all.index if i in ['helmeth2','Sabatier2',"Fischer-Tropsch2"]]].drop(['cluster','type'],axis=1).sum()
co2_stores.loc['DAC'] = co2_stores_all.loc[[i for i in co2_stores_all.index if 'DAC' in i]].drop(['cluster','type'],axis=1).sum()
co2_stores.loc['atmosphere'] = co2.loc['co2']*-1
co2_stores



,noscaling,Past_trends,LTS_Baseline,Tech,Life,Tango
CCS,486.769251,588.519414,444.781664,324.259453,219.402284,185.238406
BECCS,295.714587,295.714559,295.714590,295.714619,295.714576,181.297192
SMR CC,3.074581,0.924144,47.322649,122.988819,18.796218,124.879139
chemical production,-2.172677,-0.847957,-0.000135,-0.000029,-0.000169,-0.000013
DAC,1125.647575,1523.062142,884.570814,179.929351,176.894264,0.000007
atmosphere,230.705610,230.705610,230.705610,230.705610,230.705610,230.705610


In [204]:
n=pypsa.Network("results/Tango_DAC/postnetworks/elec_sc_Tango_s_37_lvopt__Co2L0p05-96H-T-H-B-I-A-solar+p3-dist1_2050.nc")
#n.links.loc['14811']['bus0']
ccdf=n.links.loc[[i for i in n.links.index if "gas CHP CC" in n.links.loc[i]['carrier']]]
#n.links.carrier.unique()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(ccdf.head())
#n.links_t.p0.columns

INFO:pypsa.io:Imported network elec_sc_Tango_s_37_lvopt__Co2L0p05-96H-T-H-B-I-A-solar+p3-dist1_2050.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,underwater_fraction,capital_cost,efficiency,p_nom_extendable,p_nom_min,bus2,bus3,bus4,efficiency2,efficiency3,efficiency4,lifetime,marginal_cost,p_nom_opt,charger_ratio,type,build_year,p_nom_max,p_set,p_max_pu,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AL1 0 urban central gas CHP CC,EU gas,AL1 0,urban central gas CHP CC,,0.0,0.0,,NaN,NaN,0.0,NaN,73247.998539,0.41,True,0.0,AL1 0 urban central heat,co2 atmosphere,co2 stored,0.456,0.01,0.19,25.0,4.010257,0.000261,NaN,,0,inf,0.0,1.0,1.0,NaN,NaN
AT1 0 urban central gas CHP CC,EU gas,AT1 0,urban central gas CHP CC,,0.0,0.0,,NaN,NaN,0.0,NaN,73247.998539,0.41,True,0.0,AT1 0 urban central heat,co2 atmosphere,co2 stored,0.456,0.01,0.19,25.0,4.009713,0.000303,NaN,,0,inf,0.0,1.0,1.0,NaN,NaN
BA1 0 urban central gas CHP CC,EU gas,BA1 0,urban central gas CHP CC,,0.0,0.0,,NaN,NaN,0.0,NaN,73247.998539,0.41,True,0.0,BA1 0 urban central heat,co2 atmosphere,co2 stored,0.456,0.01,0.19,25.0,4.010982,0.000274,NaN,,0,inf,0.0,1.0,1.0,NaN,NaN
BE1 0 urban central gas CHP CC,EU gas,BE1 0,urban central gas CHP CC,,0.0,0.0,,NaN,NaN,0.0,NaN,73247.998539,0.41,True,0.0,BE1 0 urban central heat,co2 atmosphere,co2 stored,0.456,0.01,0.19,25.0,4.010799,0.000321,NaN,,0,inf,0.0,1.0,1.0,NaN,NaN
BG1 0 urban central gas CHP CC,EU gas,BG1 0,urban central gas CHP CC,,0.0,0.0,,NaN,NaN,0.0,NaN,73247.998539,0.41,True,0.0,BG1 0 urban central heat,co2 atmosphere,co2 stored,0.456,0.01,0.19,25.0,4.010663,0.000269,NaN,,0,inf,0.0,1.0,1.0,NaN,NaN
